<a href="https://colab.research.google.com/github/aasem/cvisionmcs/blob/main/linearclassification_losses.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Linear Classification (SVM and Softmax) for CIFAR10 Dataset
**Preliminary Stuff - Definitions, Data Access and Visualization**


In [16]:
!git clone https://github.com/aasem/cvisionmcs
# Definitions
import numpy as np
import matplotlib.pyplot as plt
import sys
# sys.path.append('/content/cvisionmcs')
from cvisionmcs import data_utils
from cvisionmcs import download
%matplotlib inline

fatal: destination path 'cvisionmcs' already exists and is not an empty directory.


**Downloading the Dataset**

In [17]:
url = "https://www.cs.toronto.edu/~kriz/cifar-10-python.tar.gz"
download_dir = "./data"
download.maybe_download_and_extract(url,download_dir)

Data has apparently already been downloaded and unpacked.


**Converting Raw Files into Training and Test Datasets**

In [18]:
cifar10_dir = './data/cifar-10-batches-py'
X_train, y_train, X_test, y_test = data_utils.load_CIFAR10(cifar10_dir)

# Checking the size of the training and testing data
print('Training data shape: ', X_train.shape)
print('Training labels shape: ', y_train.shape)
print('Test data shape: ', X_test.shape)
print('Test labels shape: ', y_test.shape)

Training data shape:  (50000, 32, 32, 3)
Training labels shape:  (50000,)
Test data shape:  (10000, 32, 32, 3)
Test labels shape:  (10000,)


**Reshaping and Appending the Bias**

In [19]:
# reshaping data and placing into rows
X_train = np.reshape(X_train, (X_train.shape[0], -1))
X_test = np.reshape(X_test, (X_test.shape[0], -1))
print(X_train.shape, X_test.shape)

(50000, 3072) (10000, 3072)


In [20]:
# append 1 in the last column to cater for bias and transform into columns
X_train = np.append(X_train, np.ones((X_train.shape[0],1)), axis=1)
X_test = np.append(X_test, np.ones((X_test.shape[0],1)), axis=1)
X_train = np.transpose(X_train)
X_test = np.transpose(X_test)
print(X_train.shape, X_test.shape)

(3073, 50000) (3073, 10000)


**Computing SVM Loss**



In [28]:
def loss_svm(W, X, y, reg):
    """
    Compute the SVM loss.
    
    Input Parameters
    ----------
    W: (K, D) array of weights, K is the number of classes and D is the dimension of one sample.
    X: (D, N) array of training data, each column is a training sample with D-dimension.
    y: (N, ) 1-dimension array of target data with length N with lables 0,1, ... K-1, for K classes
    reg: (float) regularization strength for optimization.

    Returns
    -------
    loss: (float)
    """
    
    # initialization
    dW = np.zeros(W.shape)
    loss = 0.0
    delta = 1.0
    num_train = y.shape[0]

    # compute all scores
    scores = W.dot(X) # [C x N] matrix
 
    # get the correct class score 
    correct_class_score = scores[y, range(num_train)] # [1 x N]
    
    margins = scores - correct_class_score + delta # [C x N]

    # threshold the score to max(0, -)
    margins = np.maximum(0, margins)
    margins[y, range(num_train)] = 0

    loss = np.sum(margins) / num_train

    # add regularization to loss
    loss += 0.5 * reg * np.sum(W * W)
   
    return loss

**Computing SVM Loss**


In [29]:
def loss_softmax(W, X, y, reg):
    """ Compute the softmax loss
    
    Input Parameters
    ----------
    W: (K, D) array of weights, K is the number of classes and D is the dimension of one sample.
    X: (D, N) array of training data, each column is a training sample with D-dimension.
    y: (N, ) 1-dimension array of target data with length N with lables 0,1, ... K-1, for K classes
    reg: (float) regularization strength for optimization.
    
    Returns
    -------
    loss: (float)
    """
    loss = 0 
    grad = np.zeros_like(W)
    dim, num_train = X.shape

    scores = W.dot(X) # [K, N]
    # Shift scores so that the highest value is 0
    scores -= np.max(scores)
    scores_exp = np.exp(scores)
    correct_scores_exp = scores_exp[y, range(num_train)] # [N, ]
    scores_exp_sum = np.sum(scores_exp, axis=0) # [N, ]
    loss = -np.sum(np.log(correct_scores_exp / scores_exp_sum))
    loss /= num_train
    loss += 0.5 * reg * np.sum(W * W)

    return loss

In [30]:
import time
# generate a rand weights W 
W = np.random.randn(10, X_train.shape[0]) * 0.001
tic = time.time()
loss_svm = loss_svm(W, X_train, y_train, 0)
toc = time.time()
print('SVM loss: %f; Computed in: %fs' % (loss_svm, toc - tic))

SVM loss: 38.723339; Computed in: 0.519478s


In [31]:
# generate a rand weights W 
W = np.random.randn(10, X_train.shape[0]) * 0.001
tic = time.time()
loss_softmax = loss_softmax(W, X_train, y_train, 0.0001)
toc = time.time()
print ('Softmax loss: %f; Computed in %fs' % (loss_softmax, toc - tic))

Softmax loss: 10.346303; Computed in 0.538426s
